Link to the jupyter notebook: http://localhost:8888/notebooks/Assignment%201.ipynb

In [1]:
import numpy as np
from queue import PriorityQueue

# Class PuzzleNode: 
This class captured the state of the puzzle. A puzzle is initialized to have five attributes: state has the form of a list of list, fval and gval are used for A* search, parent is used to trace back the optimal path,  pruned, which is a boolean value that indicates whether a node is visited or not. The class has two methods: __lt__ to compare the two states based on f cost and __str__ to represent a state in a form of a grid (real puzzle) 


In [21]:
class PuzzleNode:
    # Class constructor
    def __init__(self,state,fval,gval,parent=None):
        self.state = state
        self.fval = fval
        self.gval = gval
        self.parent = parent
        self.pruned = False

    # Comparison function based on f cost
    def __lt__(self,other):
        return self.fval < other.fval

    # Represent the state of the puzzle as a grid  
    def __str__(self):
        n=len(self.state)
        grid = ''
        for row in range(n):
            grid += ' '.join(map(str, self.state[row]))
            grid += '\r\n'
        return grid


# Functions to check valid input:
I wrote two functions to check whether an input is in the right form, and whether the state is solvable. 
The function checking if a puzzle is solvable would call another function inside it to get the value of inversions. With a puzzle of size n x n
If n is odd, the puzzle is solvable if number of inversions is even 
If n is even, the puzzle is solvable if 1/ the blank (0) lies on an even row counting from the bottom and the number of inversions is odd. 2/ the blank lies on an odd row counting from the bottom and the number of inversions is even. 
When we convert the state from a 2D array (or list of list) into a 1D array and we have a pair of tiles (a, b). An inversion is counted if a appears before b but a > b. 


In [15]:
#Check input:
def check_valid(n, puzzle):
    result=True
    #Assuming the smallest possible puzzle is 3x3
    if len(puzzle) != n or len(puzzle) < 3:
        result=False
    #Check if input contains every number from 0 to n**2-1 
    flat=[i for sublist in puzzle for i in sublist]
    result=(sorted(flat) == list(range(n**2)))
    return result
    
def check_solvable(n, puzzle):
    inv_count = get_inv_count(n,puzzle)
    if n % 2 == 1:
        return (inv_count % 2 == 0)
    else:
        pos = find_blank_pos(puzzle)
        if pos % 2 == 1:
            return (inv_count % 2 == 0)
        else:
            return (inv_count % 2 == 1)
def find_blank_pos(n, puzzle):
    for i in range(n-1,-1,-1):
        for j in range(n-1,-1,-1):
            if puzzle[i][j]==0:
                return n-i
def get_inv_count(n, puzzle):
    flat=[i for sublist in puzzle for i in sublist]
    count = 0
    for i in range(0,n*n-1):
        for j in range(i+1, n*n):
            if flat[j] and flat[i] and flat[i]>flat[j]:
                count+=1
    return count 

# The Heuristics: 
I implemented three different heuristics 
1/ The number of misplaced tiles 
2/ The Manhattan distance 
3/ The linear conflict + manhattan distance. <br />
The linear conflict combined with manhattan distance was proved to outperform the other two, even though for these three particular test cases given, it does not (as shown in the last table). Two tiles a and b are in a linear conflict if they are in the same row or column, their goal positions are also in the same row or column and the goal position of one of the tiles is blocked by the other tile in that row. <br />
After having all the heuristics, I created a list of handles to the heuristics so that we can call out any of them later 

In [4]:
def misplace_count(state):
    n = len(state)
    l = list(range(0,n**2))
    goal_state=[l[i:i+n] for i in range(0,n**2,n)]
    misplace_num=0
    for i in range(n):
        for j in range(n):
            #Leave out the blank state
            if state[i][j] == 0:
                continue
            if state[i][j] != goal_state[i][j]:
                misplace_num+=1
    return misplace_num

In [5]:
def manhattan_distance(state):
    n = len(state)
    l = list(range(0,n**2))
    goal_state=[l[i:i+n] for i in range(0,n**2,n)]
    current=[]
    target=[]
    for row in range(n):
        for col in range(n):
            #Leave out the blank state
            if state[row][col] == 0:
                continue
            current.append((row,col))
            target.append((state[row][col] // n, state[row][col] % n))
    total_dist = 0 
    for k in range(n**2-1):
        total_dist += abs(target[k][0]-current[k][0]) + abs(target[k][1]-current[k][1])
    return total_dist      

In [6]:
#Linear conflict + Manhattan distance

def linear_conflict(state):
    n = len(state)
    count_manhattan = manhattan_distance(state)
    vertical_conflict=0
    for row in range(n):
        max_ver = -1
        for col in range(n):
            value = state[row][col]
            if value != 0 and (value-1)// n == row:
                if value > max_ver:
                    max_ver = value
                else:
                    vertical_conflict+=2
    horizontal_conflict=0
    for col in range(n):
        max_hor = -1
        for row in range(n):
            value = state[row][col]
            if value != 0 and value%n == col+1:
                if value > max_hor:
                    max_hor = value
                else:
                    horizontal_conflict+=2 
    return (count_manhattan + vertical_conflict + horizontal_conflict) 
    

In [8]:
heuristics = [misplace_count, manhattan_distance, linear_conflict]

# Possible moves to change state: 
The moves here are not defined for the tiles, but for the blank space. For example, if the blank is in the middle, a right move would mean that the tile on the right of it will move left. There are four possible moves in total and each of them will have different constraints (for example, if the blank is in the top row, it cannot move up, or if the blank is in the right most column, it cannot move right) 


In [20]:
def get_index_0(state):
    for i in range(len(state)):
        for j in range(len(state)): 
            if state[i][j] == 0:
                r=i
                c=j
    return (r,c)

def moveleft(this_state):
    r,c=get_index_0(this_state)
    if c!=0:
        new_state=[x[:] for x in this_state]
        new_state[r][c], new_state[r][c-1] = \
        new_state[r][c-1], new_state[r][c]
    else:
        new_state = None
    return (new_state)

def moveright(this_state):
    r,c=get_index_0(this_state)
    if c!=len(this_state)-1:
        new_state=[x[:] for x in this_state]
        new_state[r][c], new_state[r][c+1] = \
        new_state[r][c+1], new_state[r][c]
    else:
        new_state = None
    return (new_state)
def moveup(this_state):
    r,c=get_index_0(this_state)
    if r!=0:
        new_state=[x[:] for x in this_state]
        new_state[r][c], new_state[r-1][c] = \
        new_state[r-1][c], new_state[r][c]
    else:
        new_state = None
    return (new_state)
def movedown(this_state):
    r,c=get_index_0(this_state)
    if r!=len(this_state)-1:
        new_state=[x[:] for x in this_state]
        new_state[r][c], new_state[r+1][c] = \
        new_state[r+1][c], new_state[r][c]
    else:
        new_state = None
    return (new_state)

# Main function for A* search 
This function is adapted from the code provided in section 3.1 

In [10]:
def solvePuzzle(n, state, heuristic, prnt= False):
    l = list(range(0,n**2))
    #Generate the goal state
    goal = [l[i:i+n] for i in range(0,n**2,n)]
    #Starting state would be the input state. 
    start = state 
    #Check if the input is in the correct format 
    if check_valid(n, start) == False:
        return (0, 0, -1)
    #Check if the input state is solvable:
    if check_solvable(n, start) == False:
        return (0, 0, -2)
    #Create a PuzzleNode object as the starting node. 
    start_node = PuzzleNode(start, heuristic(start),0)
    #Create a dictionary that stores current cost to reach all visited nodes
    costs_db = {str(start):start_node}
    #Create a frontier in the form of a Priority Queue to maintain ordering
    frontier = PriorityQueue()
    #Insert the starting node into frontier to start expanding it. 
    frontier.put(start_node)

    # Begin A* Search
    #Expansion_counter stores the number of steps required 
    #to reach the goal state from the initial state
    expansion_counter = 0
    # Continue to explore until there is no node remaning in the frontier 
    while not frontier.empty():
    # Take the next available node from the frontier
        cur_node = frontier.get()
        if cur_node.pruned:
            continue # Skip if this node has been marked for removal

    # Check if we already reach the goal state
        if cur_node.state == goal:
        # Reconstruct the optimal path by backward tracing using the attribute parent of each node. 
            if prnt == True:
                #If we want to show the results as grids, comment out the line below:
                #optimal_path = [cur_node.__str__()]
                #The next line wil show the result as list of list: 
                optimal_path = [cur_node.state]
                while cur_node.parent:
                    #If we want to show the results as grids, comment out the lines below:
#                     c = cur_node.parent.__str__()
#                     optimal_path.append(c)
                    ##The next line wil append the parents in the form of list of list 
                    optimal_path.append((cur_node.parent).state)
                    cur_node = cur_node.parent

                optimal_path = optimal_path[::-1]
                print("Optimal Path to Goal: ")
                for node in optimal_path:
                    print(node) 
            
            return(expansion_counter, frontier.qsize(),0)
    
    # Define moving possibilities 
        moves_pos = [moveleft, moveright, moveup, movedown]
        
    # Expand the node in the orthogonal and diagonal directions
        for m in moves_pos:
            next_state = m(cur_node.state)

        # We can only expand the node resulted from a plausible move 
        #(as defined in the 4 possible move functions above)
            if next_state != None:
                gval = cur_node.gval + 1 # Tentative cost value for child

            # If the child node has already been explored: 
                if str(next_state) in costs_db:
                    if costs_db[str(next_state)].gval > gval:
                    # Mark existing value for deletion from frontier
                        costs_db[str(next_state)].pruned = True 
                    else:
        # ignore this child, since a better path has already been found previously.
                        continue 
                
                # Heuristic cost from next node to goal
                hval = heuristic(next_state) 
        
        
                # Create new node for child
                next_node = PuzzleNode(next_state, gval + hval , gval, cur_node) 
                frontier.put(next_node)

                #Mark the node as explored
                costs_db[str(next_state)] = next_node 

                    
        expansion_counter = expansion_counter + 1

In [24]:
#Function to compare heuristics on the test sets 
def comparison(heuristics):
    test1=[[5,7,6],[2,4,3],[8,1,0]]
    test2=[[7,0,8],[4,6,1],[5,3,2]]
    test3=[[2,3,7],[1,8,0],[6,5,4]]
    tests=[test1,test2,test3]
    results=[]
    #Only consider the two heuristics 
    for heur in heuristics:
        #print ("Heuristic %s: " % str(heur))
        for test in tests:
            steps, frontierSize, err = solvePuzzle(3, test, heur)
            results.append((steps, frontierSize))
    return results

In [26]:
result1 = comparison([misplace_count, manhattan_distance])

In [27]:
from tabulate import tabulate
def print_table(result):
    row1=['Num Steps 1',result[0][0],result[3][0]]
    row2=['Frontier Size 1',result[0][1], result[3][1]]
    row3=['Num Steps 2',result[1][0],result[4][0]]
    row4=['Frontier Size 2',result[1][1],result[4][1]]
    row5=['Num Steps 3',result[2][0],result[5][0]]
    row6=['Frontier Size 3',result[2][1],result[5][1]]   
    rows = [row1,row2,row3,row4,row5,row6]
    table=tabulate(rows, headers=["Test","Heuristic 1","Heuristic 2"], tablefmt='orgtbl')
    print (table)

# Comparing Misplaced Count Heuristic and Manhattan Distance Heuristic
From the table below, we can see that in all three test cases, the misplaced count heuristic required larger number of moves and larger frontier size compared to the Manhattan Distance heuristics. 

In [28]:
print_table(result1)

| Test            |   Heuristic 1 |   Heuristic 2 |
|-----------------+---------------+---------------|
| Num Steps 1     |         62935 |          1463 |
| Frontier Size 1 |         21533 |           864 |
| Num Steps 2     |         25141 |          1534 |
| Frontier Size 2 |         12031 |           897 |
| Num Steps 3     |           813 |           118 |
| Frontier Size 3 |           507 |            64 |


# Comparing the Manhattan Distance Heuristic and the Linear Conflict Heuristic 
From the table below, we can see that in these three test cases, the Linear Conflict Heuristic do not necessarily outperform Manhattan Distance Heuristic, but if we have more test cases and average the number of steps and frontier size, the results may look different. 

In [31]:
result2 = comparison([manhattan_distance, linear_conflict])

In [32]:
print_table(result2)

| Test            |   Heuristic 1 |   Heuristic 2 |
|-----------------+---------------+---------------|
| Num Steps 1     |          1463 |          2141 |
| Frontier Size 1 |           864 |          1186 |
| Num Steps 2     |          1534 |          6104 |
| Frontier Size 2 |           897 |          3466 |
| Num Steps 3     |           118 |            92 |
| Frontier Size 3 |            64 |            56 |


# References 
GeeksforGeeks. (n.d.). How to check if an instance of 15 puzzle is solvable? - GeeksforGeeks. [online] Available at: https://www.geeksforgeeks.org/check-instance-15-puzzle-solvable/ [Accessed 18 Feb. 2018]. <br />

Insight into programming algorithms. (n.d.). Implementing A-star(A*) to solve N-Puzzle. [online] Available at: https://algorithmsinsight.wordpress.com/graph-theory-2/a-star-in-general/implementing-a-star-to-solve-n-puzzle/ [Accessed 18 Feb. 2018].